### CNN Models
- Multiplication of the variables
- Convert the multiplication into grid for each data point
- Train CNN model
- Fine-tune the CNN model

In [130]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [131]:
df_cnn = pd.read_pickle('../../Data/output/df_process.pkl')

In [132]:
df_cnn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 611 entries, 2 to 735
Data columns (total 53 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Age                                                          611 non-null    float64
 1   Hours per day                                                611 non-null    float64
 2   While working                                                611 non-null    int64  
 3   Instrumentalist                                              611 non-null    int64  
 4   Composer                                                     611 non-null    int64  
 5   Exploratory                                                  611 non-null    int64  
 6   Foreign languages                                            611 non-null    int64  
 7   BPM                                                          611 non-null    float64


In [133]:
genres_freq_columns = df_cnn.iloc[:, 8:24]  # Columns 8-23
other_columns = pd.concat([df_cnn.iloc[:, :8], df_cnn.iloc[:, 24:28], df_cnn.iloc[:, 29:49]], axis=1)

target_columns = ['Music effects_Improve', 'Music effects_No effect', 'Music effects_Worsen']
target = df_cnn[target_columns]

new_columns = {}

for genre_col in genres_freq_columns.columns:
    for other_col in other_columns.columns:
        new_columns[f'{genre_col}_x_{other_col}'] = genres_freq_columns[genre_col] * other_columns[other_col]

feature_grid = pd.DataFrame(new_columns)

for col in target_columns:
    feature_grid[col] = target[col]

print(feature_grid.head())
print("Total rows in feature grid:", feature_grid.shape[0])

   Frequency [Classical]_x_Age  Frequency [Classical]_x_Hours per day   
2                    -0.000000                               0.000000  \
3                     6.202447                              -0.783636   
4                    -0.000000                               0.000000   
5                    -0.581585                               0.421143   
6                    -1.163169                              -0.458451   

   Frequency [Classical]_x_While working   
2                                      0  \
3                                      2   
4                                      0   
5                                      1   
6                                      2   

   Frequency [Classical]_x_Instrumentalist  Frequency [Classical]_x_Composer   
2                                        0                                 0  \
3                                        0                                 2   
4                                        0              

In [134]:
def find_factors_close_to_square(n):
    factors = [(i, n // i) for i in range(1, int(n**0.5) + 1) if n % i == 0]
    return sorted(factors, key=lambda x: abs(x[0]-x[1]))[0]

grid_depth = len(other_columns.columns)

if 512 % grid_depth == 0:
    sub_features = 512 // grid_depth 
    height, width = find_factors_close_to_square(sub_features)
    print(f"Suggested grid dimensions: Depth={grid_depth}, Height={height}, Width={width}")
else:
    print("Grid depth does not evenly divide into 512 features. Please reconsider grid depth.")

Suggested grid dimensions: Depth=32, Height=4, Width=4


In [135]:
X_reshaped = X.reshape(-1, 32, 4, 4)

X_tensor = torch.tensor(X_reshaped, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [136]:
for data, target in train_loader:
    print("Data shape:", data.shape) 
    break

Data shape: torch.Size([64, 32, 4, 4])


In [137]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn as nn
import torch.optim as optim

class MusicEffectCNN(nn.Module):
    def __init__(self):
        super(MusicEffectCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Processes 32 channels
            nn.ReLU(),
            nn.MaxPool2d(2)  # Results in (64, 2, 2)
        )
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 2 * 2, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 3)  # Three output classes

    def forward(self, x):
        x = self.layer1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [138]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MusicEffectCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

total_train_loss = 0
total_train_correct = 0
total_train_samples = 0
total_test_loss = 0
total_test_correct = 0
total_test_samples = 0

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * data.size(0) 
        _, predicted = torch.max(output, 1)
        total_train_correct += (predicted == target).sum().item()
        total_train_samples += data.size(0)
    
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_test_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            total_test_correct += (predicted == target).sum().item()
            total_test_samples += data.size(0)

avg_train_loss = total_train_loss / total_train_samples
train_accuracy = total_train_correct / total_train_samples
avg_test_loss = total_test_loss / total_test_samples
test_accuracy = total_test_correct / total_test_samples

print(f'Overall Training Loss: {avg_train_loss:.4f}, Overall Training Accuracy: {train_accuracy:.4f}')
print(f'Overall Test Loss: {avg_test_loss:.4f}, Overall Test Accuracy: {test_accuracy:.4f}')

Overall Training Loss: 0.3445, Overall Training Accuracy: 0.8694
Overall Test Loss: 0.8953, Overall Test Accuracy: 0.7220


#### Enhanced the complexity of the CNN model

In [116]:
class EnhancedCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU(inplace=True)
        
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU(inplace=True)
        
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU(inplace=True)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256, 1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        
        x = self.pool2(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

In [117]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EnhancedCNN(input_channels=32, num_classes=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02)
criterion = nn.CrossEntropyLoss()

total_train_loss = 0
total_train_correct = 0
total_train_samples = 0
total_test_loss = 0
total_test_correct = 0
total_test_samples = 0

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output, 1)
        total_train_correct += (predicted == target).sum().item()
        total_train_samples += data.size(0)
    
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_test_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            total_test_correct += (predicted == target).sum().item()
            total_test_samples += data.size(0)

avg_train_loss = total_train_loss / total_train_samples
train_accuracy = total_train_correct / total_train_samples
avg_test_loss = total_test_loss / total_test_samples
test_accuracy = total_test_correct / total_test_samples

print(f'Overall Training Loss: {avg_train_loss:.4f}, Overall Training Accuracy: {train_accuracy:.4f}')
print(f'Overall Test Loss: {avg_test_loss:.4f}, Overall Test Accuracy: {test_accuracy:.4f}')

Overall Training Loss: 0.1098, Overall Training Accuracy: 0.9618
Overall Test Loss: 1.0464, Overall Test Accuracy: 0.6949


#### Using the best hyperparameters from MLP model

In [98]:
class CustomCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        mlp_layers = []
        neuron_sizes = [128]*2 + [64]*2 + [32]*16
        input_size = 64 * 2 * 2
        
        for size in neuron_sizes:
            mlp_layers.append(nn.Linear(input_size, size))
            mlp_layers.append(nn.ReLU())
            input_size = size
        
        self.classifier = nn.Sequential(*mlp_layers)
        self.classifier.add_module("final_fc", nn.Linear(input_size, num_classes))
        self.dropout_layers = [0.3]*15 + [0.2]*5  # Dropout rates per layer
        
        for i, dropout_rate in enumerate(self.dropout_layers, 1):
            if dropout_rate > 0:
                self.classifier.add_module(f"dropout_{i}", nn.Dropout(dropout_rate))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [100]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN(input_channels=32, num_classes=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02)
criterion = nn.CrossEntropyLoss()

dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

total_train_loss = 0
total_train_correct = 0
total_train_samples = 0
total_test_loss = 0
total_test_correct = 0
total_test_samples = 0

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output, 1)
        total_train_correct += (predicted == target).sum().item()
        total_train_samples += data.size(0)

    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_test_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            total_test_correct += (predicted == target).sum().item()
            total_test_samples += data.size(0)

avg_train_loss = total_train_loss / total_train_samples
train_accuracy = total_train_correct / total_train_samples
avg_test_loss = total_test_loss / total_test_samples
test_accuracy = total_test_correct / total_test_samples

print(f'Overall Training Loss: {avg_train_loss:.4f}, Overall Training Accuracy: {train_accuracy:.4f}')
print(f'Overall Test Loss: {avg_test_loss:.4f}, Overall Test Accuracy: {test_accuracy:.4f}')

Overall Training Loss: 1.2172, Overall Training Accuracy: 0.7496
Overall Test Loss: 1.0653, Overall Test Accuracy: 0.5431
